# Fine-tuning Mistral-7B con QLoRA

In [1]:
!pip install transformers datasets peft bitsandbytes accelerate

In [2]:
!git clone https://github.com/xaviguardia/llm01.git && cd llm01

fatal: destination path 'llm01' already exists and is not an empty directory.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
# set the directory to /content/llm01
import os
os.chdir("/content/llm01")
model_id = "mistralai/Mistral-7B-v0.1"
dataset_path = "data/qa_dataset.jsonl"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map="auto")

model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
import os

# Load the JSON data using pandas
df = pd.read_json(dataset_path, lines=True)

# Convert the pandas DataFrame to a datasets Dataset object
dataset = Dataset.from_pandas(df)

def tokenize(sample):
    tokenized_sample = tokenizer(
        sample["prompt"] + "\n" + sample["response"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized_sample["labels"] = tokenized_sample["input_ids"].copy()
    return tokenized_sample

tokenized = dataset.map(tokenize)

args = TrainingArguments(
    output_dir="./outputs",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    fp16=True, # Descomenta si tu GPU lo soporta
)

trainer = Trainer(model=model, args=args, train_dataset=tokenized)
trainer.train()

In [ ]:
!wget https://raw.githubusercontent.com/xaviguardia/llm01/main/data/qa_dataset.jsonl -P /content/llm01/data/